In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import seaborn as sns
import os
print(os.listdir("../input"))
from tqdm import tqdm_notebook as tqdm

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)

from sklearn.metrics import accuracy_score
import warnings 
warnings.filterwarnings("ignore") 
pd.set_option('display.max_colwidth', 200)
# Any results you write to the current directory are saved as output.

['imdb_master.csv']


In [3]:
df = pd.read_csv('../input/imdb_master.csv', encoding='latin')
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the charact...",neg,0_2.txt
1,1,test,"This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and...",neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangst...",neg,10001_1.txt
3,3,test,"Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social pla...",neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass p...,neg,10003_3.txt


In [4]:
df.drop(['Unnamed: 0', 'file'], axis=1, inplace = True)

In [5]:
df.head()

,type,review,label
0,test,"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the charact...",neg
1,test,"This is an example of why the majority of action films are the same. Generic and boring, there's really nothing worth watching here. A complete waste of the then barely-tapped talents of Ice-T and...",neg
2,test,"First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangst...",neg
3,test,"Not even the Beatles could write songs everyone liked, and although Walter Hill is no mop-top he's second to none when it comes to thought provoking action movies. The nineties came and social pla...",neg
4,test,Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass p...,neg


In [6]:
df['label'].unique()

array(['neg', 'pos', 'unsup'], dtype=object)

In [7]:
df['label'].value_counts()

unsup    50000
neg      25000
pos      25000
Name: label, dtype: int64

In [8]:
df = df[df['label'] != 'unsup']

In [9]:
df['label'] = df['label'].map({'neg' : 0, 'pos' : 1})

In [10]:
train = df[df['type'] == 'train'].drop(['type'], axis= 1)
test  = df[df['type'] == 'test'].drop(['type'], axis = 1)

In [11]:
train.head()

,review,label
25000,"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob ...",0
25001,Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to...,0
25002,This film lacked something I couldn't put my finger on at first: charisma on the part of the leading actress. This inevitably translated to lack of chemistry when she shared the screen with her le...,0
25003,"Sorry everyone,,, I know this is supposed to be an ""art"" film,, but wow, they should have handed out guns at the screening so people could blow their brains out and not watch. Although the scene d...",0
25004,"When I was little my parents took me along to the theater to see Interiors. It was one of many movies I watched with my parents, but this was the only one we walked out of. Since then I had never ...",0


In [12]:
import re
def decontracted(phrase):
     # specific    
    phrase = re.sub(r"won't", "will not", phrase)    
    phrase = re.sub(r"can\'t", "can not", phrase) 
    # general    
    phrase = re.sub(r"n\'t", " not", phrase)  
    phrase = re.sub(r"\'re", " are", phrase)  
    phrase = re.sub(r"\'s", " is", phrase)    
    phrase = re.sub(r"\'d", " would", phrase)  
    phrase = re.sub(r"\'ll", " will", phrase)   
    phrase = re.sub(r"\'t", " not", phrase)   
    phrase = re.sub(r"\'ve", " have", phrase)   
    phrase = re.sub(r"\'m", " am", phrase)   
    return phrase 


In [13]:
# https://gist.github.com/sebleier/554280 
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're" , "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'th ey', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "tha t'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'ha d', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'ove r', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any' , 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'no w', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'might n', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wa sn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"] 


In [14]:
 def preprocess_text(dff, feature):   
        preprocessed_text = []   
        for sentance in tqdm(dff[feature].values):       
            sent = decontracted(sentance)      
            sent = sent.replace('\\r', ' ')     
            sent = sent.replace('\\"', ' ')     
            sent = sent.replace('\\n', ' ')     
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)      
            # https://gist.github.com/sebleier/554280     
            sent = ' '.join(e for e in sent.split() if e not in stopwords)    
            preprocessed_text.append(sent.lower().strip()) 
        return preprocessed_text 

In [15]:
train_prep_reviews = preprocess_text(train, 'review')
test_prep_reviews = preprocess_text(test, 'review')

## **Vectorizing Text Data**

**1. Bag of Words**

In [16]:
vectorizer = CountVectorizer(min_df = 10)
train_bow = vectorizer.fit_transform(train_prep_reviews)
test_bow = vectorizer.transform(test_prep_reviews)
print("Shape of train matrix after BOW : ",train_bow.shape)
print("Shape of test matrix after BOW : ",test_bow.shape) 

Shape of train matrix after BOW :  (25000, 18512)
Shape of test matrix after BOW :  (25000, 18512)


**2. Tf-Idf**

In [17]:
vectorizer = TfidfVectorizer(min_df = 10)
train_tfidf = vectorizer.fit_transform(train_prep_reviews)
test_tfidf = vectorizer.transform(test_prep_reviews)
print("Shape of train matrix after Tfidf : ",train_tfidf.shape)
print("Shape of test matrix after Tfidf : ",test_tfidf.shape) 

Shape of train matrix after Tfidf :  (25000, 18512)
Shape of test matrix after Tfidf :  (25000, 18512)


## Model on BOW

In [18]:
X_tr = train_bow
y_tr = train['label']
X_ts = test_bow
y_ts = test['label']

In [19]:
print(X_tr.shape)
print(y_tr.shape)
print(X_ts.shape)
print(y_ts.shape)

(25000, 18512)
(25000,)
(25000, 18512)
(25000,)


In [20]:
models = [RandomForestClassifier(random_state=77),
          GradientBoostingClassifier(random_state=77),
          AdaBoostClassifier(random_state=77)]

from sklearn.model_selection import cross_val_score, GridSearchCV

for model in models:
    score = cross_val_score(model, X_tr, y_tr, cv=5)
    msg = ("{0}:\n\tMean accuracy on development set\t= {1:.3f} "
           "(+/- {2:.3f})".format(model.__class__.__name__,
                                  score.mean(),
                                  score.std()))
    print(msg)
    
    # Fit the model on the dev set and predict and eval independent set
    model.fit(X_tr, y_tr)
    pred_eval = model.predict(X_ts)
    acc_eval = accuracy_score(y_ts, pred_eval)
    print("\tAccuracy on evaluation set\t\t= {0:.3f}".format(acc_eval))

RandomForestClassifier:
	Mean accuracy on development set	= 0.759 (+/- 0.004)
	Accuracy on evaluation set		= 0.770
GradientBoostingClassifier:
	Mean accuracy on development set	= 0.808 (+/- 0.003)
	Accuracy on evaluation set		= 0.810
AdaBoostClassifier:
	Mean accuracy on development set	= 0.801 (+/- 0.002)
	Accuracy on evaluation set		= 0.802


## Model on Tf-idf

In [25]:
X_tr = train_tfidf
y_tr = train['label']
X_ts = test_tfidf
y_ts = test['label']

In [26]:
print(X_tr.shape)
print(y_tr.shape)
print(X_ts.shape)
print(y_ts.shape)

(25000, 18512)
(25000,)
(25000, 18512)
(25000,)


In [27]:
models = [RandomForestClassifier(random_state=77),
          GradientBoostingClassifier(random_state=77),
          AdaBoostClassifier(random_state=77)]

from sklearn.model_selection import cross_val_score, GridSearchCV

for model in models:
    score = cross_val_score(model, X_tr, y_tr, cv=5)
    msg = ("{0}:\n\tMean accuracy on development set\t= {1:.3f} "
           "(+/- {2:.3f})".format(model.__class__.__name__,
                                  score.mean(),
                                  score.std()))
    print(msg)
    
    # Fit the model on the dev set and predict and eval independent set
    model.fit(X_tr, y_tr)
    pred_eval = model.predict(X_ts)
    acc_eval = accuracy_score(y_ts, pred_eval)
    print("\tAccuracy on evaluation set\t\t= {0:.3f}".format(acc_eval))

RandomForestClassifier:
	Mean accuracy on development set	= 0.766 (+/- 0.004)
	Accuracy on evaluation set		= 0.765
GradientBoostingClassifier:
	Mean accuracy on development set	= 0.809 (+/- 0.004)
	Accuracy on evaluation set		= 0.812
AdaBoostClassifier:
	Mean accuracy on development set	= 0.798 (+/- 0.004)
	Accuracy on evaluation set		= 0.803


In [21]:
# import time
# dict_clf = {}

In [22]:
# paramgrid = {
#     'n_estimators':      [100, 200, 500, 750, 1000],
#     'criterion':         ['gini', 'entropy'],
#     'max_features':      ['auto', 'log2'],
#     'min_samples_leaf':  list(range(2, 7))
# }
# GS = GridSearchCV(RandomForestClassifier(random_state=77),
#                   paramgrid,
#                   cv=4)
# t0 = time.time()
# GS.fit(X_tr, y_tr)
# t = time.time() - t0
# best_clf = GS.best_estimator_
# best_params = GS.best_params_
# best_score = GS.best_score_
# name = 'RF'
# best_clf.fit(X_tr, y_tr)
# acc_eval = accuracy_score(y_ts, best_clf.predict(X_ts))
# dict_clf[name] = {
#     'best_par': best_params,
#     'best_clf': best_clf,
#     'best_score': best_score,
#     'score_eval': acc_eval,
#     'fit_time': t,
# }

In [23]:
# # 2. GradientBoosting
# paramgrid = {
#     'n_estimators':      [100, 200, 500, 750, 1000],
#     'max_features':      ['auto', 'log2'],
#     'min_samples_leaf':  list(range(2, 7)),
#     'loss' :             ['deviance', 'exponential'],
#     'learning_rate':     [0.05, 0.1, 0.2],
# }
# GS = GridSearchCV(GradientBoostingClassifier(random_state=77),
#                   paramgrid,
#                   cv=4)
# t0 = time.time()
# GS.fit(X_tr, y_tr)
# t = time.time() - t0
# best_clf = GS.best_estimator_
# best_params = GS.best_params_
# best_score = GS.best_score_
# name = 'GB'
# best_clf.fit(X_tr, y_tr)
# acc_eval = accuracy_score(y_ts, best_clf.predict(X_ts))
# dict_clf[name] = {
#     'best_par': best_params,
#     'best_clf': best_clf,
#     'best_score': best_score,
#     'score_eval': acc_eval,
#     'fit_time': t,
# }

In [24]:
# # 3. AdaBoost
# paramgrid = {
#     'n_estimators':  [100, 200, 500, 750, 1000],
#     'learning_rate': [0.05, 0.1, 0.5, 1, 2]
# }
# GS = GridSearchCV(AdaBoostClassifier(random_state=77),
#                   paramgrid,
#                   cv=4)
# t0 = time.time()
# GS.fit(X_tr, y_tr)
# t = time.time() - t0
# best_clf = GS.best_estimator_
# best_params = GS.best_params_
# best_score = GS.best_score_
# name = 'ADB'
# best_clf.fit(X_tr, y_tr)
# acc_eval = accuracy_score(y_ts, best_clf.predict(X_ts))
# dict_clf[name] = {
#     'best_par': best_params,
#     'best_clf': best_clf,
#     'best_score': best_score,
#     'score_eval': acc_eval,
#     'fit_time': t,
# }